In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn import naive_bayes as NB
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical

In [2]:
input_df = pd.read_csv('data/en_cards.csv')

In [3]:
input_df.head()

,Unnamed: 0,name,lang,released_at,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,set,rarity
0,0,Fury Sliver,en,2006-10-06,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,['R'],['R'],[],tsp,uncommon
1,1,Kor Outfitter,en,2009-10-02,{W}{W},2.0,Creature — Kor Soldier,"When Kor Outfitter enters the battlefield, you...",2,2,['W'],['W'],[],zen,common
2,3,Spirit,en,2015-05-22,NaN,0.0,Token Creature — Spirit,Flying,1,1,['W'],['W'],['Flying'],tmm2,common
3,4,Siren Lookout,en,2017-09-29,{2}{U},3.0,Creature — Siren Pirate,Flying\nWhen Siren Lookout enters the battlefi...,1,2,['U'],['U'],"['Flying', 'Explore']",xln,common
4,6,Web,en,1994-04-01,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,['G'],['G'],['Enchant'],3ed,rare


In [4]:
# pandas has a habit of adding 'Unnamed: {int}' columns; maybe I'm reading in wrong?
input_df.drop('Unnamed: 0', axis = 1, inplace = True)
# reset all values to strings, because every 'document (card)' should have the same fields
input_df.fillna(value = '[none]', inplace = True)

In [5]:
# can change predicted attribute here
target = input_df.pop('cmc').values

In [6]:
input_df.apply(lambda x: x.astype(str))

,name,lang,released_at,mana_cost,type_line,oracle_text,power,toughness,colors,color_identity,keywords,set,rarity
0,Fury Sliver,en,2006-10-06,{5}{R},Creature — Sliver,All Sliver creatures have double strike.,3,3,['R'],['R'],[],tsp,uncommon
1,Kor Outfitter,en,2009-10-02,{W}{W},Creature — Kor Soldier,"When Kor Outfitter enters the battlefield, you...",2,2,['W'],['W'],[],zen,common
2,Spirit,en,2015-05-22,[none],Token Creature — Spirit,Flying,1,1,['W'],['W'],['Flying'],tmm2,common
3,Siren Lookout,en,2017-09-29,{2}{U},Creature — Siren Pirate,Flying\nWhen Siren Lookout enters the battlefi...,1,2,['U'],['U'],"['Flying', 'Explore']",xln,common
4,Web,en,1994-04-01,{G},Enchantment — Aura,Enchant creature (Target a creature as you cas...,[none],[none],['G'],['G'],['Enchant'],3ed,rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56195,Hollow One,en,2020-08-13,{5},Artifact Creature — Golem,This spell costs {2} less to cast for each car...,4,4,[],[],['Cycling'],akr,rare
56196,Angel's Tomb,en,2015-07-17,{3},Artifact,Whenever a creature enters the battlefield und...,[none],[none],[],[],[],ori,uncommon
56197,Celestine Reef,en,2009-09-04,[none],Plane — Luvion,Creatures without flying or islandwalk can't a...,[none],[none],[],[],[],phop,rare
56198,Horned Troll,en,2003-07-28,{2}{G},Creature — Troll,{G}: Regenerate Horned Troll.,2,2,['G'],['G'],[],8ed,common


In [7]:
# passing the df grabs df.columns, and we want each row as a document
corpus = []
for doc in input_df.values.tolist():
    doc_list = []
    for term in doc:
        doc_list.append(term)
    corpus.append(str(doc_list))

In [8]:
# vectorizer = CountVectorizer()
# vectorizer.fit(corpus)
# X_train_counts = vectorizer.transform(corpus)
term_vec = TfidfVectorizer()
X = term_vec.fit_transform(corpus)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, target)

In [17]:
nb_model = NB.MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nb_model.fit(X_train, y_train.astype(str))

MultinomialNB()

In [19]:
nb_model.score(X_train, y_train.astype(str))

0.6189798339264532

In [20]:
nb_model.score(X_test, y_test.astype(str))

0.5447686832740214

array([0., 5., 1., ..., 3., 6., 2.])